### Import all the libraries

In [2]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import json
import time

### Set config options for undetected chromedriver

In [3]:
options = uc.ChromeOptions()
options.headless = False
driver = uc.Chrome(options=options, use_subprocess=False)

### Bypass cloudflare protection running a url to website

In [4]:
url = "https://darulifta-deoband.com/en"
driver.get(url)

### Load all question links

In [5]:
with open("question_urls.json", "r") as question_file:
  question_links_list = json.load(question_file)

### Open all log files

In [6]:
width = 12
log_file = open("logs.txt", "a")
value_error_log = open("value_error_logs.txt", "a")
index_error_log = open("index_error_logs.txt", "a")
log_file.write("question_no | count      | i          | time_spent \n")

52

### Loop through all question links and scrap the data

- format data
- handle exceptions
- write data to log file

In [7]:
count = 0
start_time = time.time()
with open("QnA_Dataset.json", "a") as QnA_Dataset_file:
  i=0
  while i < len(question_links_list):
    question_link = question_links_list[i]
    try:
      driver.get(question_link)
      answer_detail_element = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CLASS_NAME, "answer_detail"))
      )
      if not answer_detail_element.is_displayed():
        raise Exception("cloudflare")
      li_element = answer_detail_element.find_element(By.TAG_NAME, "li")
      H2s = li_element.find_elements(By.TAG_NAME, "h2")
      Spans = li_element.find_elements(By.TAG_NAME, "span")
      Ps = li_element.find_elements(By.TAG_NAME, "p")
      QnA_structure = {
        "question_link": question_link,
        "question_id": int(Spans[2].text[13:]),
        "answer_id": int(Spans[4].text[11:]),
        "category": Spans[0].text,
        "subcategory": Spans[1].text,
        "title": H2s[0].text,
        "question": H2s[1].text,
        "answer": " ".join(p.text for p in Ps[4:len(Ps)-3]).split(")", 1)[1]
      }
    except KeyboardInterrupt:
      choice = input("Press enter to continue")
      if choice == "exit":
        break
    except IndexError as e:
      print(f"Question {i} couldnt be done as index out of range, skipping...")
      index_error_log.write(f"{question_link}\n")
    except ValueError:
      print(f"Question {i} couldnt be done as value error, skipping...")
      value_error_log.write(f"{question_link}\n")
# catch Exception("cloudflare") and retry the request
    except Exception as e:
      print(e)
      # input("Please solve the cloudflare security check and press enter to continue")
      print(f"Question {i} couldnt be done, skipping...")
    else:
      json.dump(QnA_structure,QnA_Dataset_file)
      QnA_Dataset_file.write(",\n")
      print(f"Question count:{count+1}, i:{i} done")
    finally:
      count+=1
      i+=1

    if count % 10 == 0:
      log_text = f"{question_link.split("/")[::-1][0]:>{width}}|{count:>{width}}|{i:>{width}}|{time.time()-start_time:>{width+10}.2f} \n"
      log_file.write(log_text)
      print(log_text)
    

Question count:1, i:0 done
Question count:2, i:1 done
Question count:3, i:2 done
Question count:4, i:3 done
Question count:5, i:4 done
Question count:6, i:5 done
Question count:7, i:6 done
Question 7 couldnt be done as index out of range
Question count:9, i:8 done
Question count:10, i:9 done
       32151|          10|          10|                  9.16 

Question count:11, i:10 done
Question count:12, i:11 done
Question count:13, i:12 done
Question count:14, i:13 done
Question count:15, i:14 done
Question count:16, i:15 done
Question count:17, i:16 done
Question count:18, i:17 done
Question count:19, i:18 done
Question count:20, i:19 done
        4014|          20|          20|                 18.71 

Question count:21, i:20 done
Question count:22, i:21 done
Question count:23, i:22 done
Question count:24, i:23 done
Question count:25, i:24 done
Question count:26, i:25 done
Question count:27, i:26 done
Question count:28, i:27 done
Question count:29, i:28 done
Question count:30, i:29 done

### Close files and quit driver

In [8]:
log_file.close()
value_error_log.close()
index_error_log.close()

In [9]:
driver.close()
driver.quit()